##Connecting Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import zipfile
audio = zipfile.ZipFile('/content/gdrive/MyDrive/data.zip', 'r')
audio.extractall('/content')

##Meta creation

In [ ]:
import pandas as pd
from pathlib import Path
import os
from tqdm.notebook import tqdm

data_folder = Path('/content/val') # specify train or validation folder
df = []

for path, _, files in tqdm(os.walk(data_folder)):
    for file in files:
      
      folder = path.split('/')[-3]
      type_ = path.split('/')[-2]
      id = path.split('/')[-1]
      df.append([folder, type_, id, file])

df = pd.DataFrame(df, columns = ['folder', 'type', 'id', 'path'])
df.to_csv('meta_val.csv')

0it [00:00, ?it/s]

##Meta Demonstration

In [ ]:
import pandas as pd

demonstration = pd.read_csv('/content/meta_train.csv')
demonstration.head()

,Unnamed: 0,folder,type,id,path
0,0,train,noisy,6625,6625_70995_6625-70995-0008.npy
1,1,train,noisy,6625,6625_70995_6625-70995-0004.npy
2,2,train,noisy,6625,6625_39674_6625-39674-0024.npy
3,3,train,noisy,6625,6625_39680_6625-39680-0034.npy
4,4,train,noisy,6625,6625_70995_6625-70995-0006.npy


##Sound Dataset

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import os
from tqdm.notebook import tqdm

class SoundDataset(Dataset):

    def __init__(self, meta, source_folder, transforms):

        self.source_folder = source_folder
        self.transforms = transforms
        self.meta = meta
        self.files = self.meta.path.unique()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):

        sound = self.files[i]
        sound_df = self.meta.loc[self.meta.path == sound]

        if np.random.rand() < 0.5:
            label = 0.0
            pic_row = sound_df.loc[sound_df.type == 'clean'].iloc[0]
        else:
            label = 1.0
            pic_row = sound_df.loc[sound_df.type == 'noisy'].iloc[0]


        pic_path = os.path.join(self.source_folder, pic_row.get('folder'), pic_row.get('type'), str(pic_row.get('id')), sound)

        pic = np.expand_dims(np.load(pic_path).astype(float), 2)

        augmented = self.transforms(image=pic)

        return {'pic': augmented['image'][0].unsqueeze(0).float(), 'label': torch.Tensor([label]).float()}
        

##Separation of meta: Training (80%), validation (20%)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/meta_train.csv', index_col=0) # Specify mata train

df = df.sort_values(by=['id'], ignore_index=True)

train_size = int(0.8 * df.shape[0])
val_size = df.shape[0] - train_size

border_id = df['id'][train_size]

train_df = df[df['id'] < border_id]
val_df = df[df['id'] >= border_id]

In [ ]:
!pip install pytorch_lightning
!pip install segmentation-models-pytorch

##Train

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp
import albumentations as albu
from albumentations.pytorch import ToTensorV2

class LightningLearner(pl.LightningModule):

    def __init__(self, train_df, val_df, source_folder):
        super().__init__()

        params=dict(
            pooling='avg',
            dropout=0.2,
            activation='sigmoid',
            classes=1)
        self.unet = smp.Unet(encoder_name='resnet18', in_channels=1, aux_params = params)
        self.loss = {'Accuracy': nn.BCEWithLogitsLoss()}
        self.transforms = albu.Compose([albu.PadIfNeeded(480, 80),
                          albu.RandomCrop(480, 80),
                          albu.Resize(576, 96),
                          ToTensorV2()])
        self.trainset = SoundDataset(train_df, source_folder, self.transforms)
        self.validset = SoundDataset(val_df, source_folder, self.transforms)

    def forward(self, x):
        return self.unet(x)

    def _calc_loss(self, batch):
        pic, gt_label = batch['pic'], batch['label']
        pr_image, pr_label = self(pic)
        loss = self.loss['Accuracy'](pr_label, gt_label)
        return loss

    def training_step(self, batch, batch_nb):
        loss = self._calc_loss(batch)
        self.log('train_loss', loss, prog_bar=False)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._calc_loss(batch)
        self.log('valid_loss', loss, prog_bar=False)
        return loss

    def validation_epoch_end(self, outputs):
        loss_value = torch.stack([x for x in outputs]).mean()
        self.log('valid_loss', loss_value, prog_bar=True)

    def configure_optimizers(self):
        opt = optim.Adam(self.unet.parameters(), lr=0.0001)
        sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=10)
        return [opt], [sch]

    def train_dataloader(self):
        return DataLoader(self.trainset, batch_size=32,
                          num_workers=1, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.validset, batch_size=32,
                          num_workers=1, shuffle=False)

In [ ]:
model = LightningLearner(train_df, val_df, '/content/data') # Specify data folder

trainer = pl.Trainer(
    gpus=1,
    logger=False,
    max_epochs=50,
    precision=32,
)

trainer.fit(model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type | Params
------------------------------
0 | unet | Unet | 14.3 M
------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


##Save weights

In [ ]:
import torch

torch.save(model.state_dict(), '/content/gdrive/MyDrive/unet_22.pth')

##Test

In [ ]:
!pip install pytorch_lightning
!pip install segmentation-models-pytorch
!pip install torchmetrics

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import zipfile
audio = zipfile.ZipFile('/content/gdrive/MyDrive/data.zip', 'r')
audio.extractall('/content')

In [5]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp
import albumentations as albu
from albumentations.pytorch import ToTensorV2
import pandas as pd
from pathlib import Path
import os
from tqdm.notebook import tqdm
import numpy as np
from torchmetrics import Accuracy

In [9]:
class SoundDataset(Dataset):

    def __init__(self, meta, source_folder, transforms):

        self.source_folder = source_folder
        self.transforms = transforms
        self.meta = meta
        self.files = self.meta.path.unique()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):

        sound = self.files[i]
        sound_df = self.meta.loc[self.meta.path == sound]

        if np.random.rand() < 0.5:
            label = 0.0
            pic_row = sound_df.loc[sound_df.type == 'clean'].iloc[0]
        else:
            label = 1.0
            pic_row = sound_df.loc[sound_df.type == 'noisy'].iloc[0]


        pic_path = os.path.join(self.source_folder, pic_row.get('folder'), pic_row.get('type'), str(pic_row.get('id')), sound)

        pic = np.expand_dims(np.load(pic_path).astype(float), 2)

        augmented = self.transforms(image=pic)

        return {'pic': augmented['image'][0].unsqueeze(0).float(), 'label': torch.Tensor([label]).float()}
        

In [10]:
class LightningPredictor(nn.Module):

    def __init__(self, test_df, source_folder):
        super().__init__()

        params=dict(
            pooling='avg',
            dropout=0.2,
            activation='sigmoid',
            classes=1)
        self.unet = smp.Unet(encoder_name='resnet18', in_channels=1, aux_params = params)
        self.loss = {'Accuracy': Accuracy(0.5)}
        self.transforms = albu.Compose([albu.PadIfNeeded(480, 80),
                                        albu.RandomCrop(480, 80),
                                        albu.Resize(576, 96),
                                        ToTensorV2()])
        self.testset = SoundDataset(test_df, source_folder, self.transforms)
        self.testloader = DataLoader(self.testset, batch_size=1, shuffle=False)

    def calculate_metrics(self):

        Accuracy = list()

        with torch.no_grad():
            for batch in tqdm(self.testloader):
                pic, gt_label = batch['pic'], batch['label']
                pr_image, pr_label = self.unet(pic)
                Accuracy.append(self.loss['Accuracy'](pr_label, gt_label.type(torch.int64)).detach().numpy())

        return np.mean(Accuracy)

In [11]:
test_df = pd.read_csv('/content/meta_val.csv') # Specify test meta
model = LightningPredictor(test_df, '/content/data') # Specify data folder
model.load_state_dict(torch.load('/content/gdrive/MyDrive/unet_22.pth')) # Specify weights
model.eval()
metrics = model.calculate_metrics()

  0%|          | 0/2000 [00:00<?, ?it/s]

In [13]:
print('Accuracy =', metrics)

Accuracy = 0.992
